In [182]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import workbook
from openpyxl.styles.numbers import BUILTIN_FORMATS
from openpyxl.utils import get_column_letter


In [171]:
oms = pd.read_csv('journal_20230216 (3).csv', sep=';',
                  usecols=['Статус', 'Цель', 'Тип талона', 'Пациент', 'Начало лечения', 'Окончание лечения', 'Врач', 'Врач (Профиль МП)', 'Диагноз основной (DS1)',
                           'Подразделение'])

In [172]:
# выделяем диагноз
oms_1 = oms['Диагноз основной (DS1)'].str.split(' ', expand=True)
oms['Диагноз'] = oms_1[0]
oms = oms.drop(columns = ['Диагноз основной (DS1)'],axis = 1)

In [173]:
# функиция распределения в файле ОМС корпуса
def korpus(x):
    if x in gp3:
        return 'ГП3'
    elif x in gp3_oap_1:
        return 'ГП3 ОАП 1'
    elif x in gp3_oap_2:
        return 'ГП3 ОАП 2'
    elif x in gp3_gk:
        return 'ГП3 ЖК'
    elif x in gp3_dp:
        return 'ГП3 ДП'
    elif x in gp11:
        return 'ГП11'
    elif x in gp11_dp:
        return 'ГП11 ДП'
    elif x in gp11_dp_po:
        return 'ГП11 ДП ПО'
    elif x in gp11_dp_coz:
        return 'ГП11 ДП ЦОЗ'
    else:
        return 'проверить корпус'

gp3 = ['БУЗ ВО ВГП № 3""']
gp3_oap_1 = ['Отделение амбулаторно - поликлинической помощи № 1(БУЗ ВО ВГП №3)"']
gp3_oap_2 = ['Отделение амбулаторно-поликлинической помощи № 2 (БУЗ ВО ВГП № 3")"']
gp3_gk = ['Отделение женской консультации (БУЗ ВО ВГП № 3")"']
gp3_dp = ['Детская поликлиника № 1 (БУЗ ВО ВГП № 3")"']
gp11 = ['Поликлиника № 11 (БУЗ ВО  ВГП № 3")"']
gp11_dp = ['Детская поликлиника № 8 (БУЗ ВО ВГП № 3")"']
gp11_dp_po = ['Профилактическое отделение Детской поликлиники №8 (БУЗ ВО ВГП №3")"']
gp11_dp_coz = ['Центр охраны зрения детей (БУЗ ВО ВГП № 3")"']

oms['корпус'] = oms['Подразделение'].apply(korpus)

In [174]:
svod_po_celam = pd.pivot_table(oms, index=['корпус', 'Врач (Профиль МП)', 'Врач'],
                               columns=['Цель'],
                               values=["Пациент"],
                               aggfunc=[np.mean, len],
                               fill_value= 0)

C:\Users\User\AppData\Local\Temp\ipykernel_13624\3648636161.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  svod_po_celam = pd.pivot_table(oms, index=['корпус', 'Врач (Профиль МП)', 'Врач'],


In [175]:
svod = svod_po_celam['len']['Пациент']

In [176]:
svod['посещения. цели: 1,5,7,9,10,13,14,140,64,640'] = 0
svod['обращения. цели: 30, 301, 305'] = 0
svod['неотложка. цели: 22'] = 0
svod['3 цель'] = 0
svod['Лаб. исследования. цели: 561'] = 0
svod['УЗИ и эндоскопия. цели: 541'] = 0
svod['Цель 32'] = 0
svod['Дневной стационар'] = 0
svod['Диспансеризация ДВ4'] = 0
svod['Диспансеризация ДВ2'] = 0
svod['Диспансеризация ОПВ'] = 0
svod['Диспансеризация УД1'] = 0
svod['Диспансеризация УД2'] = 0
svod['Диспансеризация ПН1'] = 0
svod['Диспансеризация ДС2'] = 0
svod['План'] = 0
svod['%'] = 0


pos = ['1', '5', '7', '9', '10', '13', '14', '140', '64', '640']
obr = ['30', '301', '305']
neo = ['22']
cel_3 = ['3']
cel_561 = ['561']
cel_541 = ['541']
cel_32 = ['32']
stac = ['В дневном стационаре', 'На дому']
dv4 = ['ДВ4']
dv2 = ['ДВ2']
opv = ['ОПВ']
ud1 = ['УД1']
ud2 = ['УД2']
pn1 = ['ПН1']
ds2 = ['ДС2']

for i in svod.columns.values.tolist():
    if i in pos:
        svod['посещения. цели: 1,5,7,9,10,13,14,140,64,640'] = svod['посещения. цели: 1,5,7,9,10,13,14,140,64,640'] + svod[str(i)]
    elif i in obr:
        svod['обращения. цели: 30, 301, 305'] = svod['обращения. цели: 30, 301, 305'] + svod[str(i)]
    elif i in neo:
        svod['неотложка. цели: 22'] = svod['неотложка. цели: 22'] + svod[str(i)]  
    elif i in cel_3:
        svod['3 цель'] = svod['3 цель'] + svod[str(i)]
    elif i in cel_561:
        svod['Лаб. исследования. цели: 561'] = svod['Лаб. исследования. цели: 561'] + svod[str(i)]
    elif i in cel_541:
        svod['УЗИ и эндоскопия. цели: 541'] = svod['УЗИ и эндоскопия. цели: 541'] + svod[str(i)]
    elif i in cel_32:
        svod['Цель 32'] = svod['Цель 32'] + svod[str(i)]
    elif i in stac:
        svod['Дневной стационар'] = svod['Дневной стационар'] + svod[str(i)]
    elif i in dv4:
        svod['Диспансеризация ДВ4'] = svod['ДВ4'] + svod[str(i)]    
    elif i in dv2:
        svod['Диспансеризация ДВ2'] = svod['ДВ2'] + svod[str(i)]
    elif i in opv:
        svod['Диспансеризация ОПВ'] = svod['ОПВ'] + svod[str(i)]
    elif i in ud1:
        svod['Диспансеризация УД1'] = svod['УД1'] + svod[str(i)]
    elif i in ud2:
        svod['Диспансеризация УД2'] = svod['УД2'] + svod[str(i)]
    elif i in pn1:
        svod['Диспансеризация ПН1'] = svod['ПН1'] + svod[str(i)]
    elif i in ds2:
        svod['Диспансеризация ДС2'] = svod['ДС2'] + svod[str(i)]
        

In [177]:
svod = svod[['План',
                'посещения. цели: 1,5,7,9,10,13,14,140,64,640',
             '%',
             'План',
                'обращения. цели: 30, 301, 305',
             '%',
             'План',
                'неотложка. цели: 22',
             '%',
             'План',
                '3 цель',
             '%',
             'План',
                'Лаб. исследования. цели: 561',
             '%',
             'План',
                'УЗИ и эндоскопия. цели: 541',
             '%',
             'План',
                'Цель 32',
             '%',
             'План',
                'Дневной стационар',
             '%',
                'Диспансеризация ДВ4',
                'Диспансеризация ДВ2',
                'Диспансеризация ОПВ',
                'Диспансеризация УД1',
                'Диспансеризация УД2',
                'Диспансеризация ПН1',
                'Диспансеризация ДС2',
                '1',
                '10',
                '13',
                '17',
                '2',
                '22',
                '3',
                '30',
                '301',
                '306',
                '32',
                '5',
                '52',
                '541',
                '55',
                '561',
                '64',
                '7',
                'В дневном стационаре',
                'На дому'
]]

In [178]:
svod.to_excel('otch3.xlsx')

In [179]:
wb = openpyxl.load_workbook('otch3.xlsx')  # Открываем файл EXCEL
sheet = wb['Sheet1']  # Открываем нужную книгу в файле

In [180]:
for y in range(2, 23, 3):
    for i in range(2, row_count+1):
        sheet[chr(68 + y)+ str(i)] = '='+chr(68+y-1) + str(i) + '/' +chr(68+y-2) + str(i) + '*100'

In [191]:
sheet.column_dimensions['B'].width = 30
sheet.column_dimensions['C'].width = 30

In [193]:
sheet.insert_rows(1)

In [202]:
y = 0
chr(68 + y) + '1:' + chr(68 + y + 2)+'1'

'D1:F1'

In [203]:
for y in range(0, 23, 3):
    sheet.merge_cells(chr(68 + y) + '1:' + chr(68 + y + 2)+'1')

ValueError: Y1:[1 is not a valid coordinate or range

In [196]:
wb.save('otch3.xlsx')

In [198]:
for y in range(0,23, 3):
    print(y+2)

2
5
8
11
14
17
20
23
